## Generate simulation data   

In [1]:
#https://toruseo.jp/UXsim/docs/index.html

from uxsim import *
import itertools
import random
seed = None

W = World(
    name="",
    deltan=5,
    tmax=3600, #1 hour simulation
    print_mode=1, save_mode=0, show_mode=1,
    random_seed=seed,
    duo_update_time=600
)
random.seed(seed)

# network definition
"""
    N1  N2  N3  N4 
    |   |   |   |
W1--I1--I2--I3--I4-<E1
    |   |   |   |
    v   ^   v   ^
    S1  S2  S3  S4
"""

signal_time = 20
sf_1=1
sf_2=1

I1 = W.addNode("I1", 1, 0, signal=[signal_time*sf_1,signal_time*sf_2])
I2 = W.addNode("I2", 2, 0, signal=[signal_time*sf_1,signal_time*sf_2])
I3 = W.addNode("I3", 3, 0, signal=[signal_time*sf_1,signal_time*sf_2])
I4 = W.addNode("I4", 4, 0, signal=[signal_time*sf_1,signal_time*sf_2])
W1 = W.addNode("W1", 0, 0)
E1 = W.addNode("E1", 5, 0)
N1 = W.addNode("N1", 1, 1)
N2 = W.addNode("N2", 2, 1)
N3 = W.addNode("N3", 3, 1)
N4 = W.addNode("N4", 4, 1)
S1 = W.addNode("S1", 1, -1)
S2 = W.addNode("S2", 2, -1)
S3 = W.addNode("S3", 3, -1)
S4 = W.addNode("S4", 4, -1)

#E <-> W direction: signal group 0
for n1,n2 in [[W1, I1], [I1, I2], [I2, I3], [I3, I4], [I4, E1]]:
    W.addLink(n2.name+n1.name, n2, n1, length=500, free_flow_speed=50, jam_density=0.2, number_of_lanes=3, signal_group=0)
    
#N -> S direction: signal group 1
for n1,n2 in [[N1, I1], [I1, S1], [N3, I3], [I3, S3]]:
    W.addLink(n1.name+n2.name, n1, n2, length=500, free_flow_speed=30, jam_density=0.2, signal_group=1)

#S -> N direction: signal group 2
for n1,n2 in [[N2, I2], [I2, S2], [N4, I4], [I4, S4]]:
    W.addLink(n2.name+n1.name, n2, n1, length=500, free_flow_speed=30, jam_density=0.2, signal_group=1)
    

# random demand definition every 30 seconds
dt = 30
demand = 2 #average demand for the simulation time
demands = []
for t in range(0, 3600, dt):
    dem = random.uniform(0, demand)
    for n1, n2 in [[N1, S1], [S2, N2], [N3, S3], [S4, N4]]:
        W.adddemand(n1, n2, t, t+dt, dem*0.25)
        demands.append({"start":n1.name, "dest":n2.name, "times":{"start":t,"end":t+dt}, "demand":dem})
    for n1, n2 in [[E1, W1], [N1, W1], [S2, W1], [N3, W1],[S4, W1]]:
        W.adddemand(n1, n2, t, t+dt, dem*0.75)
        demands.append({"start":n1.name, "dest":n2.name, "times":{"start":t,"end":t+dt}, "demand":dem})

W.exec_simulation()
W.analyzer.print_simple_stats()
W.analyzer.basic_to_pandas()

simulation setting:
 scenario name: 
 simulation duration:	 3600 s
 number of vehicles:	 14775 veh
 total road length:	 6500 m
 time discret. width:	 5 s
 platoon size:		 5 veh
 number of timesteps:	 720
 number of platoons:	 2955
 number of links:	 13
 number of nodes:	 14
 setup time:		 0.07 s
simulating...
      time| # of vehicles| ave speed| computation time
       0 s|        0 vehs|   0.0 m/s|     0.00 s


/home/dimitris/miniconda3/envs/condapy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


     600 s|      560 vehs|   4.9 m/s|     0.41 s
    1200 s|      570 vehs|   4.7 m/s|     0.88 s
    1800 s|      555 vehs|   3.7 m/s|     1.31 s
    2400 s|      575 vehs|   4.3 m/s|     1.73 s
    3000 s|      570 vehs|   4.5 m/s|     2.12 s
    3600 s|      575 vehs|   2.3 m/s|     2.49 s
 simulation finished
results:
 average speed:	 8.3 m/s
 number of completed trips:	 6920 / 14775
 average travel time of trips:	 924.5 s
 average delay of trips:	 884.1 s
 delay ratio:			 0.956
 total distance traveled:	 12178916.7 m


,total_trips,completed_trips,total_travel_time,average_travel_time,total_delay,average_delay
0,14775,6920,6397775.0,924.53396,6118025.0,884.107659


## Connect to kafka broker and create topic “vehicle_positions”

In [2]:
# first set up the and run the kafka server also install kafka-python. The commands can be found in 'kakfa commands.txt'

from kafka import KafkaAdminClient
from kafka.admin import NewTopic

# Create an instance of KafkaAdminClient
admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092",  # Update with your Kafka broker(s)
    client_id='my_client'
)

topic_name = 'vehicle_positions'
num_partitions = 1
replication_factor = 1
topic_configs = {"retention.ms": str(60000), # all messages that are older than 10 min in the topic get deleted
                 "retention.bytes": str(int(100e6)), #when the messages in the topic exide 100 MB old messages get deleted 
                 'max.message.bytes': str(int(5e6))} #max allowed size in MB of each message set to 5MB 

try:
    my_first_topic = NewTopic(name=topic_name, num_partitions=num_partitions,
                            replication_factor=replication_factor,
                            topic_configs=topic_configs)

    admin_client.create_topics(new_topics=[my_first_topic])
    print(f'created topic: {topic_name}')
except:
    pass
    print(f'topic: {topic_name} already existed')
    
# List all topics
topics = admin_client.list_topics()
print("Topics:", topics)


topic: vehicle_positions already existed
Topics: ['my_first_topic', 'vehicle_positions', '__consumer_offsets']


## Start sending data to kafka topic

In [ ]:
from datetime import datetime, timedelta
import time
import random

current_time = datetime.now()

from kafka import KafkaProducer
import json
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))
df = W.analyzer.vehicles_to_pandas()

delay_data = []
for t in range(0,3600, 5):
    filtered_df = df[(df['t'] == t) & (df['link'] != 'waiting_at_origin_node') & (df['link'] != 'trip_end')]
    if len(filtered_df) !=0: print(f'\n #### Sending data for time {t} seconds #### \n')
    
    for data in delay_data:
        producer.send(topic_name, value=data, partition=0)# the data is sent to the topic, in a specific partition , if the topic dosent exist it is created 
        print(data)
    delay_data = []
    
    for row in filtered_df.itertuples():
        vehicle_time = current_time + timedelta(seconds=row.t)
        vehicle_time = vehicle_time.strftime("%d/%m/%Y %H:%M:%S")
        data = {'name':row.name, 'origin':row.orig, 'destination': row.dest, 'time': vehicle_time, 'link':row.link, 'position':row.x, 'spacing':row.s, 'speed':row.v}
        
        if random.randint(0,9) ==0:
            delay_data.append(data)
            continue
        
        producer.send(topic_name, value=data, partition=0)# the data is sent to the topic, in a specific partition , if the topic dosent exist it is created 
        print(data)
    time.sleep(5)#sleep time can be modified if we want to send data faster, sparks watermark will work right because it takes into accoutn not the current time of the host machine but of the timespamp of the latest data send

producer.flush()# block until all async messages are sent 
producer.close()# close the connection

In [5]:
admin_client.delete_topics(topics=[topic_name])

DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='vehicle_positions', error_code=0)])